In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [2]:
model = load_model('BRM.h5')
test_filenames = os.listdir(".../Ungrouped/test/")
TEST_PATH = ".../Ungrouped/test/"
BATCH_SIZE=64

In [3]:
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [4]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    TEST_PATH, 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(32,32),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    shuffle=False)

Found 11800 validated image filenames.


In [5]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/BATCH_SIZE))
test_df['character'] = np.argmax(predict, axis=-1)

In [6]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['character'] = test_df['character'].replace(label_map)

NameError: name 'train_generator' is not defined

In [ ]:
test_df["character"] = test_df["character"].replace({ 0:'a',1:'b', 2:'ba',3:'bi',4:'bo',5:'d',6:'da',7:'di',8:'do',9:'e',10:'g',
                                         11:'ga',12:'gi',13:'gu',14:'h',15:'ha',16:'hi',17:'ho',18:'k',19:'ka',20:'ki',
                                         21:'ko',22:'l',23:'la',24:'li',25:'lu',26:'m',27:'ma',28:'mi',29:'mu',30:'n',
                                         31:'na',32:'ni',33:'ng',34:'nga',35:'ngi',36:'ngu',37:'no',38:'o',39:'p',40:'pa',
                                         41:'pi',42:'pu',43:'s',44:'sa',45:'si',46:'su',47:'t',48:'ta',49:'ti',50:'tu',
                                         51:'w',52:'wa',53:'wi',54:'wo',55:'y',56:'ya',57:'ye',58:'yo'})

In [ ]:
test_df['character'].value_counts().plot.bar(figsize=(16,5))

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['character']
    img = load_img(TEST_PATH+filename, target_size=(32,32))
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['character']
submission_df.drop(['filename', 'character'], axis=1, inplace=True)
submission_df.to_csv('test-results.csv', index=False)